In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import copy
import matplotlib.pyplot as plt

Customer = pd.read_csv('/Users/alex/Desktop/CustomerClustering.csv')
# Customer = Customer.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
v1 = Customer['Visit.Time'].values  # variable 1
v2 = Customer['Average.Expense'].values  # variable 2
v3 = Customer['Age'].values  # variable 3
X = np.array(list(zip(v1, v2, v3)))

# Define Chebyshev Distance

In [2]:
def dist(l, m, ax):
    return np.linalg.norm(l - m, ord=np.inf, axis=ax)  # The meaning of axis=1 can be find on P.S

In [3]:
np.random.seed(26)
k = 3
C_x = np.random.choice(v1, size=k)
C_y = np.random.choice(v2, size=k)
C_z = np.random.choice(v3, size=k)
C = np.array(list(zip(C_x, C_y, C_z)), dtype=np.float32)
print(C)
# To store the value of centroids when it updates
C_old = np.zeros(C.shape)
# Clustering Lables
clusters = np.zeros(len(X))
# Error func. - Distance between new centroids and old centroids
error = dist(C, C_old, None)

# 5. Run KMeans and doing random initialization
# Loop will run till the error becomes zero
lost_function = np.zeros([8, 1])

for a in range(8):
    np.random.seed(a)
    k = 3
    C_x = np.random.choice(v1, size=k)
    # Y coordinates of random centroids
    C_y = np.random.choice(v2, size=k)
    # Z coordinates of random centroids
    C_z = np.random.choice(v3, size=k)
    C = np.array(list(zip(C_x, C_y, C_z)), dtype=np.float32)
    error = 1  # if error = 0, then the while loop won't execute
    while error != 0:  # if error = 0, means that all the centroids are the same as the previous
        # Assigning each value to its closest cluster
        for i in range(len(X)):
            distances = dist(X[i], C, 1)
            cluster = np.argmin(distances)  # There is an explanation of how it works
            clusters[i] = cluster
        # Storing the old centroid values
        C_old = copy.deepcopy(C)  # There is an explanation of the meaning of deepcopy
        # Finding the new centroids by taking the average value
        for i in range(k):
            points = [X[j] for j in range(len(X)) if clusters[j] == i]
            C[i] = np.mean(points, axis=0)
        error = dist(C, C_old, None)
    for b in range(len(X)):
        if clusters[b] == 0:
            c = [dist(X[b], C[0], 0)]
        elif clusters[b] == 1:
            c = [dist(X[b], C[1], 0)]
        else:
            c = [dist(X[b], C[2], 0)]
        lost_function[a] += c


np.random.seed(np.argmin(lost_function))  # Use the centroids to run again K-Means
k = 3
C_x = np.random.choice(v1, size=k)
C_y = np.random.choice(v2, size=k)
C_z = np.random.choice(v3, size=k)
C = np.array(list(zip(C_x, C_y, C_z)), dtype=np.float32)
error = 1  # if error = 0, means that all the centroids are the same as the previous
while error != 0:
    # Assigning each value to its closest cluster
    for i in range(len(X)):
        distances = dist(X[i], C, 1)
        cluster = np.argmin(distances)  # There is an explanation of how it works
        clusters[i] = cluster
    # Storing the old centroid values
    C_old = copy.deepcopy(C)  # There is an explanation of the meaning of deepcopy
    # Finding the new centroids by taking the average value
    for i in range(k):
        points = [X[j] for j in range(len(X)) if clusters[j] == i]
        C[i] = np.mean(points, axis=0)
    error = dist(C, C_old, None)


[[ 8.  14.5 16. ]
 [12.  10.6 11. ]
 [ 7.  20.4 36. ]]


In [4]:
Customer = pd.read_csv('/Users/alex/Desktop/CustomerClustering.csv')
Type0 = Customer[clusters == 0].drop('ID', 1)  # Here we delete column 'ID' since it can't offer any information
Type1 = Customer[clusters == 1].drop('ID', 1)
Type2 = Customer[clusters == 2].drop('ID', 1)
print('Type0 length: %d; Type1 length: %d; Type2 length: %d' %((len(Type0)), (len(Type1)), (len(Type2))))

Type0 length: 10; Type1 length: 31; Type2 length: 19


## Cluster 0

In [5]:
Type0.mean()  # Type0: 成員皆在30歲以上

Visit.Time         11.30
Average.Expense    27.43
Sex                 0.70
Age                37.00
dtype: float64

In [6]:
pd.value_counts(Type0['Age'], ascending=False)
pd.pivot_table(Type0, index=['Sex'], values=['Visit.Time', 'Average.Expense', 'Age', 'Sex'],
               aggfunc={'Visit.Time': np.mean, 'Average.Expense': np.mean,
                        'Age': np.mean, 'Sex': pd.value_counts})

,Age,Average.Expense,Sex,Visit.Time
Sex,,,,
0,32.666667,30.166667,3,13.333333
1,38.857143,26.257143,7,10.428571


## Cluster 1

In [7]:
Type1.mean()  # Type1: 成員年齡分佈範圍從8歲至22歲皆有，平均花費金額為10美元

Visit.Time          5.225806
Average.Expense    10.193548
Sex                 0.741935
Age                15.000000
dtype: float64

In [8]:
pd.value_counts(Type1['Age'], ascending=False)
pd.pivot_table(Type1, index=['Sex'], values=['Visit.Time', 'Average.Expense', 'Age', 'Sex'],
               aggfunc={'Visit.Time': np.mean, 'Average.Expense': np.mean,
                        'Age': np.mean, 'Sex': pd.value_counts})

,Age,Average.Expense,Sex,Visit.Time
Sex,,,,
0,14.25000,10.700000,8,4.750000
1,15.26087,10.017391,23,5.391304


## Cluster 2

In [9]:
Type2.mean()  # Type2: 成員年齡分佈範圍從16歲至30歲皆有，平均花費金額為22美元

Visit.Time         12.052632
Average.Expense    22.800000
Sex                 0.578947
Age                23.736842
dtype: float64

In [10]:
pd.value_counts(Type2['Age'], ascending=False)
pd.pivot_table(Type2, index=['Sex'], values=['Visit.Time', 'Average.Expense', 'Age', 'Sex'],
               aggfunc={'Visit.Time': np.mean, 'Average.Expense': np.mean,
                        'Age': np.mean, 'Sex': pd.value_counts})

,Age,Average.Expense,Sex,Visit.Time
Sex,,,,
0,23.500000,21.050000,8,11.375000
1,23.909091,24.072727,11,12.545455


In [11]:
Customer['Cluster'] = clusters
Customer.sort_values(by=['Age', 'Cluster'], ascending=[True, True])

,ID,Visit.Time,Average.Expense,Sex,Age,Cluster
29,30,1,4.5,1,8,1.0
34,35,1,4.5,1,8,1.0
40,41,3,8.3,1,8,1.0
14,15,4,17.4,0,9,1.0
0,1,3,5.7,0,10,1.0
58,59,6,8.1,1,10,1.0
9,10,3,5.3,0,11,1.0
26,27,6,9.1,1,11,1.0
50,51,3,5.3,1,11,1.0
22,23,3,5.6,1,12,1.0


In [12]:
Customer.sort_values(by=['Cluster', 'Average.Expense'], ascending=[True, True])

,ID,Visit.Time,Average.Expense,Sex,Age,Cluster
19,20,8,20.4,1,39,0.0
56,57,13,25.2,1,47,0.0
32,33,13,25.5,1,36,0.0
59,60,11,26.3,1,45,0.0
36,37,10,27.6,1,41,0.0
6,7,12,28.5,0,33,0.0
12,13,12,28.5,0,33,0.0
27,28,9,29.4,1,32,0.0
45,46,9,29.4,1,32,0.0
2,3,16,33.5,0,32,0.0
